In [ ]:
import tushare as ts
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
ts.set_token('808bf3dd5d9ecbad0130ffc842aa3338112ddbb389e91fa967240921')
pro = ts.pro_api()
import time

import datetime
import warnings
warnings.filterwarnings("ignore")

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei')  # 解决Seaborn中文显示问题

In [12]:
class datamanage():
#     管理股票的价格数据，从tushare获取数据，如果已经有数据，就不获取。

    def dbinit(self):
        self.dbconn=pymysql.connect(host='10.0.1.210',port=3306,user='root',password='751982THUNDERlz',database='leizquant')
        self.dbcur=self.dbconn.cursor()
        self.dbcur.execute('''create table if not exists stocksdaily (ts_code varchar(20),trade_date date,openprice float,highprice float,	
                           lowprice float,closeprice float,pre_closeprice float,changeprice float,pct_chg float,vol float,amount float,
                           primary key(ts_code,trade_date))''')
        self.dbcur.execute('''create table if not exists stocklist (ts_code varchar(20) primary key,symbol	varchar(20),
                            name varchar(20),area varchar(20),industry varchar(20),list_date date)''')
        
    def __init__(self,startdate='20090101',enddate=datetime.datetime.today().strftime('%Y%m%d'),stocks=[]):
        self.startdate=startdate
        self.enddate=enddate
        self.stocks=stocks
        self.dbinit()
#         获取股票列表
        self.stocklist=pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
#         time.sleep(2)
        
#         将股票列表加入数据库
        for i in range(len(self.stocklist)):
            self.dbcur.execute('insert ignore into stocklist(ts_code,symbol,name,area,industry,list_date) values(%s,%s,%s,%s,%s,%s)'
                                ,(self.stocklist.at[i,'ts_code'],self.stocklist.at[i,'symbol'],self.stocklist.at[i,'name']
                                ,self.stocklist.at[i,'area'],self.stocklist.at[i,'industry'],self.stocklist.at[i,'list_date']))
        
        self.dbconn.commit()
    
    def __del__(self):
        self.dbconn.close
        
    def getstocksdaily(self):
        if len(self.stocks)==0:
            self.stocks=list(self.stocklist['ts_code'])
        
#         获取所有数据
        for stock in self.stocks:
            print('now is:{}'.format(stock))
            time.sleep(1)
            try:
                stockdaily=pro.daily(ts_code=stock, start_date=self.startdate, end_date=self.enddate)
            except:
                print(stock,',is error')
                break
            for i in range(len(stockdaily)):
                self.dbcur.execute('''insert ignore into stocksdaily(ts_code,trade_date,openprice,highprice,lowprice,closeprice,pre_closeprice,changeprice,pct_chg,vol,amount) 
                                    values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''',
                                   (stockdaily.at[i,'ts_code'],stockdaily.at[i,'trade_date'],float(stockdaily.at[i,'open']),float(stockdaily.at[i,'high']),
                                   float(stockdaily.at[i,'low']),float(stockdaily.at[i,'close']),float(stockdaily.at[i,'pre_close']),float(stockdaily.at[i,'change']),
                                   float(stockdaily.at[i,'pct_chg']),float(stockdaily.at[i,'vol']),float(stockdaily.at[i,'amount'])))
#         提交数据库
            self.dbconn.commit()
    
#     复权因子
    def getadjfactor(self):
        self.dbcur.execute('create table if not exists adj_factor (ts_code varchar(20),trade_date date,adj_factor float,primary key(ts_code,trade_date))')
        
        if len(self.stocks)==0:
            self.stocks=list(self.stocklist['ts_code'])
        
#         获取所有数据
        for stock in self.stocks:
            print('now is:{}'.format(stock))
            time.sleep(1)
            stockdaily=pro.adj_factor(ts_code=stock,start_date=self.startdate, end_date=self.enddate)
            for i in range(len(stockdaily)):
                self.dbcur.execute('''insert ignore into adj_factor(ts_code,trade_date,adj_factor) 
                                    values(%s,%s,%s)''',
                                   (stockdaily.at[i,'ts_code'],stockdaily.at[i,'trade_date'],float(stockdaily.at[i,'adj_factor'])))
    #         提交数据库
            self.dbconn.commit()

        
        

In [13]:
dm=datamanage()

In [14]:
dm.getstocksdaily()

now is:000001.SZ
now is:000002.SZ
now is:000004.SZ
now is:000005.SZ
now is:000006.SZ
now is:000007.SZ
now is:000008.SZ
now is:000009.SZ
now is:000010.SZ
now is:000011.SZ
now is:000012.SZ
now is:000014.SZ
now is:000016.SZ
now is:000017.SZ
now is:000018.SZ
now is:000019.SZ
now is:000020.SZ
now is:000021.SZ
now is:000022.SZ
now is:000023.SZ
now is:000025.SZ
now is:000026.SZ
now is:000027.SZ
now is:000028.SZ
now is:000029.SZ
now is:000030.SZ
now is:000031.SZ
now is:000032.SZ
now is:000034.SZ
now is:000035.SZ
now is:000036.SZ
now is:000037.SZ
now is:000038.SZ
now is:000039.SZ
now is:000040.SZ
now is:000042.SZ
now is:000043.SZ
now is:000045.SZ
now is:000046.SZ
now is:000048.SZ
now is:000049.SZ
now is:000050.SZ
now is:000055.SZ
now is:000056.SZ
now is:000058.SZ
now is:000059.SZ
now is:000060.SZ
now is:000061.SZ
now is:000062.SZ
now is:000063.SZ
now is:000065.SZ
now is:000066.SZ
now is:000068.SZ
now is:000069.SZ
now is:000070.SZ
now is:000078.SZ
now is:000088.SZ
now is:000089.SZ
now is:000090.

In [15]:
dm.getadjfactor()

now is:000001.SZ
now is:000002.SZ
now is:000004.SZ
now is:000005.SZ
now is:000006.SZ
now is:000007.SZ
now is:000008.SZ
now is:000009.SZ
now is:000010.SZ
now is:000011.SZ
now is:000012.SZ
now is:000014.SZ
now is:000016.SZ
now is:000017.SZ
now is:000018.SZ
now is:000019.SZ
now is:000020.SZ
now is:000021.SZ
now is:000022.SZ
now is:000023.SZ
now is:000025.SZ
now is:000026.SZ
now is:000027.SZ
now is:000028.SZ
now is:000029.SZ
now is:000030.SZ
now is:000031.SZ
now is:000032.SZ
now is:000034.SZ
now is:000035.SZ
now is:000036.SZ
now is:000037.SZ
now is:000038.SZ
now is:000039.SZ
now is:000040.SZ
now is:000042.SZ
now is:000043.SZ
now is:000045.SZ
now is:000046.SZ
now is:000048.SZ
now is:000049.SZ
now is:000050.SZ
now is:000055.SZ
now is:000056.SZ
now is:000058.SZ
now is:000059.SZ
now is:000060.SZ
now is:000061.SZ
now is:000062.SZ
now is:000063.SZ
now is:000065.SZ
now is:000066.SZ
now is:000068.SZ
now is:000069.SZ
now is:000070.SZ
now is:000078.SZ
now is:000088.SZ
now is:000089.SZ
now is:000090.

In [10]:
dm.dbconn.close()